In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
import os
directory = "/content/dataset"
if any(os.listdir(directory)):
  print('dataset exists')
else:
  !unzip /content/dataset -d /content/

unzip:  cannot find or open /content/dataset, /content/dataset.zip or /content/dataset.ZIP.


In [36]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from utility import LoadDataIot
from IOT_CNN import DFNet_Dropout  #DFNet_Add_Layer
import random
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [37]:
def representative_data_gen_Open():
  for input_value in tf.data.Dataset.from_tensor_slices(X_open).batch(1).take(1000):
    # Model has only one input so each data point has one element.
    yield [input_value]

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3))
    else:
        return str(size)


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

In [4]:
!python dataset/IoT_Class_filter.py

2024-02-14 23:20:06.400199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 23:20:06.400260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 23:20:06.401496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 23:20:07.695598: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Data dimensions:
X: Training data's shape :  (41898, 475)
X: Validating data's shape :  (10774, 475)
X: Testing data's shape :  (7183, 475)
X: open data's shape :  (86886, 475)


In [57]:
NB_EPOCH = 40   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 70 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 475 # Packet sequence length


OPTIMIZER = Adamax(learning_rate=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # Optimizer

NB_CLASSES = 40 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

random.seed(0)

print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = LoadDataIot()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('int8')
y_valid = y_valid.astype('int8')
y_test = y_test.astype('int8')

# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

# Convert class vectors to categorical classes matrices
y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

Number of Epoch:  40
Loading and preparing data for training, and evaluating the model
Data dimensions:
X: Training data's shape :  (41898, 475)
y: Training data's shape :  (41898,)
X: Validation data's shape :  (10774, 475)
y: Validation data's shape :  (10774,)
X: Testing data's shape :  (7183, 475)
y: Testing data's shape :  (7183,)


In [ ]:
model = DFNet_Dropout.build(input_shape=INPUT_SHAPE, nb_classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

filepath = '/content/trained_models/IoT.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Start training
history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

#Start evaluating model with testing data

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/IoT_without_softmax.hdf5')
model_penultimate.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

score_test = model_penultimate.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])

del model


In [ ]:
def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

Block=1000
X_train_Rep,y_train_Rep=shuffle(X_train, y_train)
X_test = np.load("/content/gdrive/My Drive/SydneyDatasets/IoT/X_test_5.npy")
y_test = np.load("/content/gdrive/My Drive/SydneyDatasets/IoT/y_test_5.npy")
y_train=np.argmax(y_train, axis=1)
y_valid=np.argmax(y_valid, axis=1)
#y_test=np.argmax(y_test, axis=1)

model_penultimate=load_model('/content/trained_models/IoT_without_softmax.hdf5')
model_penultimate.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])

X_test = X_test.astype('float32')

TF_LITE_MODEL_FILE_NAME = "tf_lite_model_fullint.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model(model_penultimate)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]       # COnvert to 16 bit
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8
converter.supported_ops=tf.lite.OpsSet.TFLITE_BUILTINS_INT8


tflite_model = converter.convert()

tflite_model_name = "/content/trained_models/"+TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

# print("Original Model Size:",convert_bytes(get_file_size('/content/trained_models/IoT_without_softmax.hdf5'), "KB"),"KB")
# print("Quantized model size:",convert_bytes(get_file_size(tflite_model_name), "KB"),"KB")
# print()

#X_test=np.squeeze(X_test, axis=-1)
interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))       #change the input shape according to the test dataset dimenssions
interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale,input_zero_point  = input_details[0]["quantization"]
print(input_scale,input_zero_point)


for i in range(len(X_test)):
  X_test[i] = X_test[i]/input_scale + input_zero_point

X_test=X_test[:,:,np.newaxis]
X_test = X_test.astype('int8')
tflite_model_predictions_test=[]

for i in range(5):
  interpreter.set_tensor(input_details[0]['index'], X_test[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions_test=Mod_Prediction
  else:
    tflite_model_predictions_test=np.concatenate((tflite_model_predictions_test,Mod_Prediction),axis=0)
  print(i,"Done")

acc_ = accuracy_score(np.argmax(tflite_model_predictions_test,axis=1), y_test[:len(tflite_model_predictions_test)])
print('Test accuracy TFLITE model :', acc_)

del X_test

In [58]:
dataset_dir= "/content/gdrive/My Drive/SydneyDatasets/IoT/"
X_train = np.load(dataset_dir+'X_train_5.npy')
X_valid = np.load(dataset_dir+'X_valid_5.npy')


X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=[NB_CLASSES]*len(y__open)
X_open,y_open=shuffle(X_open, y_open)

X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_open = X_open.astype('float32')

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name]=np.array([0] * NB_CLASSES)

#X_train=np.squeeze(X_train, axis=-1)
for i in range(len(X_train)):
  X_train[i] = X_train[i] / input_scale + input_zero_point

X_train = X_train[:, :,np.newaxis]
X_train = X_train.astype('int8')

Block=1000

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
interpreter.allocate_tensors()

Block=1000
tflite_model_predictions = []
for i in range(5):
  interpreter.set_tensor(input_details[0]['index'], X_train[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions=Mod_Prediction
  else:
    tflite_model_predictions=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")

del X_train
y_train=np.argmax(y_train,axis=1)

count=[0]*NB_CLASSES
txt_O = "Mean_{Class1:.0f}"
Means={}
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=np.array([0]*NB_CLASSES)

for i in range(len(tflite_model_predictions)):
  k=np.argmax(tflite_model_predictions[i])
  if (np.argmax(tflite_model_predictions[i])==y_train[i]):
    Means[txt_O.format(Class1=y_train[i])]=Means[txt_O.format(Class1=y_train[i])]+tflite_model_predictions[i]
    count[y_train[i]]+=1
print("Mean vectors Calculated")

Mean_Vectors=[]
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]
  Mean_Vectors.append(Means[txt_O.format(Class1=i)])


Mean_vectors = np.array(Mean_Vectors)
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(len(X_valid)):
  X_valid[i] = X_valid[i] / input_scale + input_zero_point

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
interpreter.allocate_tensors()

input_scale,input_zero_point  = input_details[0]["quantization"]

X_valid = X_valid[:, :,np.newaxis]
X_valid = X_valid.astype('int8')

tflite_model_predictions = []
for i in range(5):
  interpreter.set_tensor(input_details[0]['index'], X_valid[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions=Mod_Prediction
  else:
    tflite_model_predictions=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")

del X_valid
print("Complete")

0 Done
1 Done
2 Done
3 Done
4 Done
Counts:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 113, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<ipython-input-58-553c7315b6de>:65: RuntimeWarning: invalid value encountered in divide
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]


0 Done
1 Done
2 Done
3 Done
4 Done


In [ ]:
y_valid= np.argmax(y_valid,axis=1)
Values={}
for i in range(NB_CLASSES):
  Values[i]=[0]*NB_CLASSES

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])

    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        Values[y_valid[i]][k]+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist

Indexes=[]
for i in range(NB_CLASSES):
  Indexes.append([])

for i in range(NB_CLASSES):
  Tot=0
  for l in range(20):
      Min=min(Values[i])
      Tot+=Min
      Indexes[i].append(Values[i].index(Min))
      Values[i][Values[i].index(Min)]=1000000

Indexes=np.array(Indexes)

np.save('/content/temp_variables/Values.npy',np.array(Values))
np.save('/content/temp_variables/Indexes.npy',Indexes)


############## Calculating the 90% Thresholds for each method ##################

txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Distances[txt_1.format(Class1=y_valid[i])].append(dist)

#print(Distances)
Threasholds_1=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  Threasholds_1[j]=Dist[int(len(Dist)*0.90)]


np.save('/content/temp_variables/Threasholds_1.npy', Threasholds_1)

print("Thresholds are calulated for method 1")

############################################################################################
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=(np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist)
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.12)]

Threasholds_2=np.array(TH)
np.save('/content/temp_variables/Threasholds_2.npy', Threasholds_2)

print("Thresholds are calulated for method 2")

############################################################################################
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])

    Tot=dist/Tot
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.88)]

Threasholds_3=np.array(TH)
np.save('/content/temp_variables/Threasholds_3.npy', Threasholds_3)

print("Thresholds are calulated for method 3")

In [ ]:
############################################################################################

######################## Evaluating results ##########################

X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=np.array([NB_CLASSES]*len(y__open))

X_open = X_open.astype('float32')
y_open = y_open.astype('int8')

interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))
interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale,input_zero_point  = input_details[0]["quantization"]

for i in range(len(X_open)):
  X_open[i] = X_open[i] / input_scale + input_zero_point

X_open = X_open[:, :,np.newaxis]
X_open = X_open.astype('int8')

tflite_model_predictions_open = []
for i in range(5):
  interpreter.set_tensor(input_details[0]['index'], X_open[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions_open=Mod_Prediction
  else:
    tflite_model_predictions_open=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")


In [ ]:
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):

    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_test[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model_penultimate.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):

    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

y_test=y_test[:len(prediction_classes)]
y_open=y_open[:len(prediction_classes_open)]

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
print("Micro")
#print('Test accuracy TFLITE model_Closed_set :', acc_Close)
#print('Test accuracy TFLITE model_Open_set :', acc_Open)
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
#print('Test accuracy TFLITE model_Closed_set :', acc_Close)
#print('Test accuracy TFLITE model_Open_set :', acc_Open)
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)




#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)



#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

tflite_model_predictions_open = model.predict(X_open)
prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)